In [6]:
import pulp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List, Any

In [7]:
# Advanced Supply Chain Optimization Model
class AdvancedSupplyPlanningModel:
    def __init__(self):
        """Initialize the advanced supply chain optimization model."""
        self.model = pulp.LpProblem("Advanced_Supply_Chain_Optimization", pulp.LpMinimize)
        self.periods = []
        self.products = []
        self.facilities = []
        self.variables = {}
        self.constraints = []
        
    def add_facilities(self, facilities: List[str]):
        """Add production and distribution facilities."""
        self.facilities = facilities
        
    def add_products(self, products: List[str]):
        """Add products to the model."""
        self.products = products
        
    def add_periods(self, periods: List[int]):
        """Add planning periods to the model."""
        self.periods = periods
        
    def setup_variables(self, initial_inventory: Dict[str, Dict[str, float]] = None):
        """
        Set up decision variables for a comprehensive supply chain model.
        Includes production, transportation, inventory, backlog, and workforce variables.
        """
        if initial_inventory is None:
            initial_inventory = {f: {p: 0 for p in self.products} for f in self.facilities}
        
        # Production variables per facility and product
        self.variables['production'] = pulp.LpVariable.dicts(
            "production", 
            ((f, p, t) for f in self.facilities for p in self.products for t in self.periods), 
            lowBound=0
        )
        
        # Setup variables
        self.variables['setup'] = pulp.LpVariable.dicts(
            "setup", 
            ((f, p, t) for f in self.facilities for p in self.products for t in self.periods), 
            cat='Binary'
        )
        
        # Inter-facility transportation variables
        self.variables['transport'] = pulp.LpVariable.dicts(
            "transport", 
            ((f1, f2, p, t) for f1 in self.facilities for f2 in self.facilities 
             for p in self.products for t in self.periods if f1 != f2), 
            lowBound=0
        )
        
        # Workforce assignment variables
        self.variables['workforce'] = pulp.LpVariable.dicts(
            "workforce", 
            ((f, s, t) for f in self.facilities for s in ['skilled', 'unskilled'] for t in self.periods), 
            lowBound=0, 
            cat='Integer'
        )
        
        # Hiring and firing variables
        self.variables['hire'] = pulp.LpVariable.dicts(
            "hire", 
            ((f, s, t) for f in self.facilities for s in ['skilled', 'unskilled'] for t in self.periods), 
            lowBound=0, 
            cat='Integer'
        )
        
        self.variables['fire'] = pulp.LpVariable.dicts(
            "fire", 
            ((f, s, t) for f in self.facilities for s in ['skilled', 'unskilled'] for t in self.periods), 
            lowBound=0, 
            cat='Integer'
        )
        
        # Inventory and backlog variables
        self.variables['inventory'] = pulp.LpVariable.dicts(
            "inventory", 
            ((f, p, t) for f in self.facilities for p in self.products for t in self.periods), 
            lowBound=0
        )
        
        self.variables['backlog'] = pulp.LpVariable.dicts(
            "backlog", 
            ((f, p, t) for f in self.facilities for p in self.products for t in self.periods), 
            lowBound=0
        )
        
        # Overtime variables
        self.variables['overtime'] = pulp.LpVariable.dicts(
            "overtime", 
            ((f, t) for f in self.facilities for t in self.periods), 
            lowBound=0
        )
        
        self.initial_inventory = initial_inventory
        
    def add_demand_satisfaction_constraints(self, demand: Dict[str, Dict[int, float]]):
        """
        Add demand satisfaction constraints across facilities.
        Allows inventory transfers and backlogging to meet demand.
        """
        for t in self.periods:
            for p in self.products:
                # Total supply must meet or exceed demand
                self.model += (
                    pulp.lpSum(
                        self.variables['inventory'][f, p, t] - 
                        self.variables['backlog'][f, p, t] 
                        for f in self.facilities
                    ) >= demand[p][t]
                )
        
    def add_material_requirements_constraints(self, bom: Dict[str, Dict[str, float]]):
        """
        Add bill of materials (BOM) constraints.
        Tracks material consumption for each product's production.
        """
        for f in self.facilities:
            for t in self.periods:
                for material, requirement in bom.items():
                    # Constraint ensuring material availability
                    self.model += (
                        pulp.lpSum(
                            requirement[p] * self.variables['production'][f, p, t] 
                            for p in self.products
                        ) <= requirement['capacity']
                    )
        
    def add_workforce_constraints(self, workforce_params: Dict):
        """
        Add workforce management constraints.
        Includes hiring, firing, skill mix, and training requirements.
        """
        for f in self.facilities:
            for t in self.periods:
                # Workforce size constraints
                self.model += (
                    self.variables['workforce'][f, 'skilled', t] >= 
                    workforce_params['min_skilled']
                )
                
                # Hiring and firing limits
                self.model += (
                    self.variables['hire'][f, 'skilled', t] <= 
                    workforce_params['max_hire']
                )
                
                # Skill mix requirement
                self.model += (
                    self.variables['workforce'][f, 'skilled', t] >= 
                    workforce_params['skill_mix_ratio'] * 
                    pulp.lpSum(
                        self.variables['workforce'][f, s, t] 
                        for s in ['skilled', 'unskilled']
                    )
                )
        
    def set_objective_function(self, cost_parameters: Dict):
        """
        Set a comprehensive objective function minimizing total supply chain costs.
        Includes production, inventory, transportation, workforce, and backlog costs.
        """
        self.model += (
            # Production costs
            pulp.lpSum(
                cost_parameters['production_cost'][f][p] * 
                self.variables['production'][f, p, t]
                for f in self.facilities for p in self.products for t in self.periods
            ) +
            # Setup costs
            pulp.lpSum(
                cost_parameters['setup_cost'][f][p] * 
                self.variables['setup'][f, p, t]
                for f in self.facilities for p in self.products for t in self.periods
            ) +
            # Transportation costs
            pulp.lpSum(
                cost_parameters['transport_cost'][f1][f2] * 
                self.variables['transport'][f1, f2, p, t]
                for f1 in self.facilities for f2 in self.facilities 
                for p in self.products for t in self.periods if f1 != f2
            ) +
            # Inventory holding costs
            pulp.lpSum(
                cost_parameters['inventory_cost'][f][p] * 
                self.variables['inventory'][f, p, t]
                for f in self.facilities for p in self.products for t in self.periods
            ) +
            # Backlog costs
            pulp.lpSum(
                cost_parameters['backlog_cost'][f][p] * 
                self.variables['backlog'][f, p, t]
                for f in self.facilities for p in self.products for t in self.periods
            ) +
            # Workforce costs
            pulp.lpSum(
                cost_parameters['workforce_cost'][f][s] * 
                self.variables['workforce'][f, s, t]
                for f in self.facilities for s in ['skilled', 'unskilled'] for t in self.periods
            ) +
            # Hiring and firing costs
            pulp.lpSum(
                (cost_parameters['hire_cost'] * self.variables['hire'][f, s, t] +
                 cost_parameters['fire_cost'] * self.variables['fire'][f, s, t])
                for f in self.facilities for s in ['skilled', 'unskilled'] for t in self.periods
            )
        )
    
    def solve(self):
        """Solve the optimization model."""
        status = self.model.solve()
        return pulp.LpStatus[status]
    
    def get_results(self) -> pd.DataFrame:
        """Extract and transform optimization results."""
        # Detailed results extraction logic here...
        pass
    
    def generate_scenario_comparison(self, scenarios):
        """
        Run multiple scenarios and compare results.
        Scenarios are dictionaries of model parameters.
        """
        scenario_results = {}
        
        for name, scenario_params in scenarios.items():
            # Reset and configure model for each scenario
            self.__init__()
            # Apply scenario-specific configurations
            # Run optimization
            # Store results
            scenario_results[name] = self.get_results()
        
        return scenario_results
    
    def visualization_suite(self, results):
        """
        Comprehensive visualization of supply chain dynamics.
        Multiple plots covering different aspects of the optimization.
        """
        # Multiple visualization methods here
        pass

def create_sample_supply_chain_scenario():
    """
    Generate a comprehensive sample scenario for demonstration.
    """
    model = AdvancedSupplyPlanningModel()
    
    # Define facilities, products, periods
    facilities = ['Factory1', 'Factory2', 'Warehouse1']
    products = ['ProductA', 'ProductB']
    periods = list(range(6))
    
    model.add_facilities(facilities)
    model.add_products(products)
    model.add_periods(periods)
    
    # Additional scenario setup would follow...
    return model

# Example usage and demonstration
if __name__ == "__main__":
    print("Creating a sample supply chain scenario...")
    model = create_sample_supply_chain_scenario()
    # Further configuration and solving would be demonstrated here
    model.solve()

Creating a sample supply chain scenario...
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/sgromme/source/sp_constraints/.venv/lib/python3.12/site-packages/pulp/apis/../solverdir/cbc/linux/i64/cbc /tmp/7e609b0988844e288b6f19bbc6c5b812-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/7e609b0988844e288b6f19bbc6c5b812-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 5 COLUMNS
At line 7 RHS
At line 8 BOUNDS
At line 10 ENDATA
Problem MODEL has 0 rows, 1 columns and 0 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Empty problem - 0 rows, 1 columns and 0 elements
Optimal - objective value 0
Optimal objective 0 - 0 iterations time 0.002
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.01

